<a href="https://colab.research.google.com/github/rajeshpandit107/AccDNN/blob/master/ResNet18_on_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import torch
import torchvision
from torch.utils.data import Dataset
import torch.optim as optim
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.hub import load_state_dict_from_url
from torch.nn import Module
from torch import nn

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.RandomHorizontalFlip(), transforms. RandomCrop(32, padding=4)])
data_set = torchvision.datasets.CIFAR10(root='.', train=True, download=True, transform=transform)
data_loader = torch.utils.data.DataLoader(data_set, batch_size=256, shuffle=True)


Extracting ./cifar-10-python.tar.gz to .


In [ ]:
	device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
	print(f'Using {device} device.')

Using cuda:0 device.


In [ ]:
model = models.resnet18()

In [ ]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0001)

In [ ]:
model.to(device)

print(f"Started Training...")

for epoch in range(1, 100+1):
  running_loss = 0.0
  for batch_num, data in enumerate(data_loader, 0):
    inputs, labels = data[0].to(device), data[1].to(device)
    # print(inputs.shape)

    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    with torch.no_grad():
                running_loss += loss.item()
  
  print(f"Epoch {epoch} : Avg Batch Loss = {running_loss/256}")

Started Training...
Epoch 1 : Avg Batch Loss = 1.6695609991438687
Epoch 2 : Avg Batch Loss = 1.1943630180321634
Epoch 3 : Avg Batch Loss = 1.0429907939396799
Epoch 4 : Avg Batch Loss = 0.9316753996536136
Epoch 5 : Avg Batch Loss = 0.8456111445557326
Epoch 6 : Avg Batch Loss = 0.7699679678771645
Epoch 7 : Avg Batch Loss = 0.727769234450534
Epoch 8 : Avg Batch Loss = 0.6798334512859583
Epoch 9 : Avg Batch Loss = 0.6427998286671937
Epoch 10 : Avg Batch Loss = 0.6071516324300319
Epoch 11 : Avg Batch Loss = 0.5801843712106347
Epoch 12 : Avg Batch Loss = 0.5588393267244101
Epoch 13 : Avg Batch Loss = 0.5317980526015162
Epoch 14 : Avg Batch Loss = 0.5131267972756177
Epoch 15 : Avg Batch Loss = 0.4937553690979257
Epoch 16 : Avg Batch Loss = 0.4807378954719752
Epoch 17 : Avg Batch Loss = 0.4710645176237449
Epoch 18 : Avg Batch Loss = 0.4575325147015974
Epoch 19 : Avg Batch Loss = 0.4351973693119362
Epoch 20 : Avg Batch Loss = 0.4263696842826903
Epoch 21 : Avg Batch Loss = 0.42083810770418495
Ep

In [ ]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])
test_data_set = torchvision.datasets.CIFAR10(root='.', train=False, download=True, transform=transform)
test_data_loader = torch.utils.data.DataLoader(test_data_set, batch_size=256, shuffle=True)

Files already downloaded and verified


In [ ]:
correct = 0
total = 0
with torch.no_grad():
  for data in test_data_loader:
    inputs, labels = data[0].to(device), data[1].to(device)
    outputs = model(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
print(f"Accuracy: {100 * correct / total}")

Accuracy: 81.66


In [ ]:
print(f"{correct}/{total}")

8166/10000


In [ ]:
torch.save({'epoch': 100,'model_state_dict': model.state_dict(),'optimizer_state_dict': optimizer.state_dict()}, "resnet18_cifar10_pretrained_model")